In [3]:
import matplotlib.pyplot as plt
import pandas 
import math
import numpy as np
import random
from numpy import linalg
from numpy import matrix
%matplotlib inline

In [4]:
regret_info = {}
data = open('data10', 'r')
for line in data:
    user = line.split('|')[1]
    article = line.split('|')[0].split(' ')[1]
    click = int(line.split('|')[0].split(' ')[2])
    user_article = article #(user, article)
    if user_article in regret_info:
        show_info = regret_info[user_article]
        show_info[0] = show_info[0] + 1
        show_info[1] = show_info[1] + click
        regret_info[user_article] = show_info
    else:
        regret_info[user_article] = [1, click]
data.close()

Compute the 

In [5]:
for elem in regret_info.keys():
    array = regret_info[elem]
    regret_info[elem] = float(array[1])/float(array[0])

In [6]:
len(regret_info)

39

Creating hahs tables: article -> martix for article A, article -> vector for article b and 
article -> payoff estimate p.

In [7]:
matrix_A = {}
vector_b = {}
payoff = {}
features = [0.,0.,0.,0.,0.]

LinUCB algorithm!

In [8]:
alpha = 0.5
max_regret = max(regret_info.values())
print max_regret

0.077441869584


In [ ]:
data = open('data10')
user_articles = [''] * 20
article_features = {}
payoff_for_user = [0.] * 20
features = [0.,0.,0.,0.,0.]
average_distance = []

for line in data:
    user = line.split('|')[1]
    
    for i in xrange(20):
        article = line.split('|')[i + 2].split(' ')[0]
        user_articles[i] = article #(user, article)
        
        if article not in article_features:
            for j in xrange(5):
                features[j] = float(line.split('|')[i + 2].split(' ')[j + 1].split(':')[1])
            article_features[article] = features
        
        if user_articles[i] not in matrix_A:
            matrix_A[user_articles[i]] = np.identity(5)
            vector_b[user_articles[i]] = np.zeros(5)
        inverse_matrix = np.matrix(matrix_A[user_articles[i]]).I
        teta = inverse_matrix * np.matrix(vector_b[user_articles[i]]).T
        features = np.array(features)
        
        payoff[user_articles[i]] = teta.T * np.matrix(article_features[article]).T  + alpha * math.sqrt(np.matrix(article_features[article]) * inverse_matrix  * np.matrix(article_features[article]).T)
        
        payoff_for_user[i] = (float(payoff[user_articles[i]]))
    
    #break
    max_payoff = np.amax(payoff_for_user)
    average_distance.append(max_payoff - max_regret)
    winner = np.argwhere(payoff_for_user == max_payoff)
    chose_article = user_articles[random.choice(map(int, winner))]
    #print 'chose', chose_article
    if chose_article in regret_info:
        real_payoff = regret_info[chose_article]
    else:
        print 'No information about payoff of', chose_article
        real_payoff = 0.
    A = np.matrix(matrix_A[chose_article])
    b = np.matrix(vector_b[chose_article])
    matrix_A[chose_article] = A + np.matrix(article_features[chose_article]).T * np.matrix(article_features[chose_article])
    vector_b[chose_article] = b + real_payoff * np.matrix(article_features[chose_article])
data.close()

In [ ]:
plt.hist(regret_info.values())


In [ ]:
payoff_for_user